<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

# **Introduction**

The ESO Science Archive Facility (SAF) is one of the largest ground-based astronomical data repositories in the world. It contains data from ESO telescopes at La Silla, Paranal, as well as the submillimeter APEX telescope on Llano de Chajnantor and the ALMA array. The archive includes all raw data from La Silla Paranal Observatory, the corresponding calibrations, and a wide range of processed data products, either generated by ESO or contributed by the community. It also hosts selected external datasets, such as raw WFCAM/UKIDSS data from the UKIRT facility in Hawaii.

As of **May 2025**, the archive hosts over **4.6 million** science products, including:

- [3.8 million products from La Silla, Paranal, and APEX ](https://archive.eso.org/scienceportal/home?observatory=%22La%20Silla%20Paranal%20APEX%22) 
- [700 000 ALMA products](https://archive.eso.org/scienceportal/home?observatory=ALMA)  
- [2.3 million spectra](https://archive.eso.org/scienceportal/home?dp_type=SPECTRUM) (1D extracted, fully calibrated)  
- [900 000 imaging products](https://archive.eso.org/scienceportal/home?dp_type=IMAGE) in optical, near-IR, and sub-mm bands  
- [500 000 spectral cubes](https://archive.eso.org/scienceportal/home?dp_type=CUBE) from IFU and radio/mm instruments  
- [700 000 catalogue records](https://archive.eso.org/scienceportal/home?dp_type=CATALOG)

# **Exploring ESO Catalogues with Photometry and Positional Data**

This notebook is designed to help explore and query the **ESO TAP service** to identify astronomical catalogues that include **source identifiers, celestial coordinates**, and **photometric measurements with associated errors**. 

Using ADQL (Astronomical Data Query Language), we search the `TAP_SCHEMA` metadata to find catalogue tables that provide:
- A main source identifiers across different catalogues (e.g., `ID`, `NAME`, `SOURCE_ID`, etc.)
- Right Ascension and Declination
- Measurement uncertainties (e.g., positional or photometric errors)
- Photometric measurements (e.g., fluxes, magnitudes, etc.)

This helps narrow down catalogues that are suitable for precise source matching, photometric analysis, or cross-matching with external datasets.
<hr style="border:2px solid #0281c9"> </hr>

# **Imports**

In [ ]:
import numpy as np

from astropy import table
from astropy.table import vstack
from astropy.coordinates import SkyCoord
from astropy.units import Quantity

from pyvo.dal import tap

import matplotlib.pyplot as plt

ESO_TAP_CAT = "http://archive.eso.org/tap_cat"

tapcat = tap.TAPService(ESO_TAP_CAT)

# **Query for Available (Most Recent) Tables**

In the following, we query the ESO TAP service metadata to discover which catalogues (tables) are currently available for use. We focus on retrieving **only the most recent version** of each table using the `TAP_SCHEMA.tables` metadata. This ensures that any outdated or superseded catalogue versions are excluded. This is helpful for identifying the latest published datasets containing astronomical or photometric values, along with their metadata descriptions, before running more targeted data queries.

In [2]:
query = """
        SELECT schema_name, collection, table_name, title, version -- you could add: ,description, publication_date, number_rows, rel_descr_url, etc.
        FROM TAP_SCHEMA.tables
        WHERE cat_id in 
                    (select t1.cat_id cat_id 
                    from TAP_SCHEMA.tables t1
                    left outer join TAP_SCHEMA.tables t2 on (t1.title = t2.title and t1.version< t2.version) 
                    where t2.title is null)
        ORDER BY schema_name, collection, title

        -- Note: this query returns the most recent version of any catalog.
        -- Previous versions of the catalogs might also be available, see next query example (cts1b).
        -- Full explanation:
        -- All TAP services must support a set of (meta) tables, in a schema named TAP_SCHEMA.
        -- These meta-tables describe all tables and columns published through the TAP service.
        -- The meta-tables can be queried themselves via TAP.
        -- Users can discover ESO-published tables or columns by querying the various tables in the TAP_SCHEMA.
        """

result = tapcat.search(query).to_table()
result

schema_name,collection,table_name,title,version
object,object,object,object,int32
safcat,1101.A-0127,AMUSED_MAIN_SOURCE_CAT_V1,AMUSED: the MUSE Hubble Ultra-Deep Field surveys,1
safcat,195.B-0283,GNS_catalogue_V1,GALACTICNUCLEUS Survey,1
safcat,196.D-0214,EREBOS_RV_cat_fits_V1,EREBOS_RV_cat.fits,1
safcat,196.D-0214,EREBOS_cat_fits_V1,EREBOS_cat.fits,1
safcat,AMBRE,AMBRE_V1,Atmospheric Parameters and Chemical Abundances from Stellar Spectra,1
safcat,AMBRE,AMBRE_HARPS_V1,Atmospheric Parameters and Chemical Abundances from Stellar Spectra - HARPS,1
safcat,AMBRE,AMBRE_UVES_V1,Atmospheric Parameters and Chemical Abundances from Stellar Spectra - UVES,1
safcat,ATLASGAL,ATLASGAL_V1,ATLASGAL - APEX Large Area Survey of the Galaxy,1
safcat,FDS,FDS_SourceCatalogue_V1,Fornax Deep Survey Source Catalogue,1


# **Query for Tables Containing Astrometry and Uncertainties**

In the following, we query the ESO TAP service metadata to discover which tables contain **astrometric information and associated uncertainties**. Specifically, we look for columns with the following UCDs (Unified Content Descriptors):

- `meta.id;meta.main`: Main source identifier
- `pos.eq.ra;meta.main`: Right Ascension (RA)
- `pos.eq.dec;meta.main`: Declination (Dec)
- `stat.error;pos.eq.ra`: Uncertainty in RA
- `stat.error;pos.eq.dec`: Uncertainty in Dec

We first identify all columns with any of these UCDs, and then filter for those tables that contain **all five** descriptors — indicating the presence of a complete set of positional and error metadata for astronomical sources.

In [3]:
query = """
        SELECT collection, T.table_name, column_name, ucd
        FROM TAP_SCHEMA.columns as C, TAP_SCHEMA.tables as T
        WHERE T.table_name=C.table_name
        AND (ucd = 'meta.id;meta.main' or 
            ucd = 'pos.eq.ra;meta.main' or 
            ucd = 'pos.eq.dec;meta.main' or 
            ucd = 'stat.error;pos.eq.ra' or 
            ucd = 'stat.error;pos.eq.dec')

        -- The ucd tells which columns are carrying the main coordinates and source identifier.
        """

result = tapcat.search(query).to_table()
result

collection,table_name,column_name,ucd
object,object,object,object
AMBRE,AMBRE_V1,DP_ID,meta.id;meta.main
AMBRE,AMBRE_V1,RAJ2000,pos.eq.ra;meta.main
AMBRE,AMBRE_V1,DEJ2000,pos.eq.dec;meta.main
VST-ATLAS,atlas_er3_ugriz_catMetaData_fits_V2,SOURCEID,meta.id;meta.main
VST-ATLAS,atlas_er3_ugriz_catMetaData_fits_V2,RA2000,pos.eq.ra;meta.main
VST-ATLAS,atlas_er3_ugriz_catMetaData_fits_V2,DEC2000,pos.eq.dec;meta.main
ATLASGAL,ATLASGAL_V1,ATLAS_NAME,meta.id;meta.main
ATLASGAL,ATLASGAL_V1,RA,pos.eq.ra;meta.main
ATLASGAL,ATLASGAL_V1,DE,pos.eq.dec;meta.main


Now filter for those tables that contain **all five** descriptors — indicating the presence of a complete set of positional and error metadata for astronomical sources.

In [4]:
result_masked = []
table_names = np.unique(result['table_name'])
for table_name in table_names:
    result_masked_ = result[result['table_name'] == table_name]
    if len(result_masked_) >= 5:
        result_masked.append(result_masked_)
    
result_masked = vstack(result_masked)
result_masked

collection,table_name,column_name,ucd
object,object,object,object
VVVX,VVVX_VIRAC_V2_REJECTED_SOURCES,sourceid,meta.id;meta.main
VVVX,VVVX_VIRAC_V2_REJECTED_SOURCES,ra,pos.eq.ra;meta.main
VVVX,VVVX_VIRAC_V2_REJECTED_SOURCES,ra_error,stat.error;pos.eq.ra
VVVX,VVVX_VIRAC_V2_REJECTED_SOURCES,de,pos.eq.dec;meta.main
VVVX,VVVX_VIRAC_V2_REJECTED_SOURCES,de_error,stat.error;pos.eq.dec
VVVX,VVVX_VIRAC_V2_SOURCES,sourceid,meta.id;meta.main
VVVX,VVVX_VIRAC_V2_SOURCES,ra,pos.eq.ra;meta.main
VVVX,VVVX_VIRAC_V2_SOURCES,ra_error,stat.error;pos.eq.ra
VVVX,VVVX_VIRAC_V2_SOURCES,de,pos.eq.dec;meta.main


Find the names of columns with the UCDs `meta.id;meta.main`, `pos.eq.ra;meta.main`, `pos.eq.dec;meta.main`, `stat.error;pos.eq.ra`, and `stat.error;pos.eq.dec` for a catalogue table of interest (e.g. `VVVX_VIRAC_V2_SOURCES`)

In [5]:
result_table  = result[result["table_name"] == "VVVX_VIRAC_V2_SOURCES"]

column_id = result_table[result_table["ucd"] == "meta.id;meta.main"]["column_name"][0]
column_ra = result_table[result_table["ucd"] == "pos.eq.ra;meta.main"]["column_name"][0]
column_dec = result_table[result_table["ucd"] == "pos.eq.dec;meta.main"]["column_name"][0]
column_ra_err = result_table[result_table["ucd"] == "stat.error;pos.eq.ra"]["column_name"][0]
column_dec_err = result_table[result_table["ucd"] == "stat.error;pos.eq.dec"]["column_name"][0]

Now get the data for this table. 

In [6]:
query = f"""
        SELECT {column_id}, {column_ra}, {column_dec}, {column_ra_err}, {column_dec_err}
        FROM VVVX_VIRAC_V2_SOURCES
        """

# result = tapcat.search(query, maxrec=100000).to_table() # to increase the number of rows, use maxrec=100000 or higher
result_table_data = tapcat.search(query).to_table()
result_table_data

/opt/anaconda3/envs/tst/lib/python3.9/site-packages/pyvo/dal/query.py:341: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


sourceid,ra,de,ra_error,de_error
,deg,deg,mas,mas
int64,float64,float64,float64,float64
13550551016169,266.4084876728242,-29.149677083154415,8.046338335725856,7.383418271984562
13550551006294,266.45124917744147,-29.165269592578042,9.570640695196355,8.100599445030499
13550551008796,266.43861747586493,-29.143605072366665,5.838562117760776,4.45382184878046
13550551000268,266.4103850308817,-29.14851364733599,6.5778333943788505,6.2211105022193625
13550551013147,266.43465092359696,-29.171138624298848,9.933415976376736,9.44183225709199
13550551005638,266.4653569344437,-29.14858784072673,7.502170093296878,6.82829031931015
13550551005394,266.46160925899335,-29.142507185473548,3.743919010199219,4.780539055122571
13550551005200,266.4720624656265,-29.147478268333966,13.58794958371984,12.583253966421948


# **Query a Specific Table Containing Photometric Information**

In this section, we query the ESO TAP service metadata to identify columns containing **photometric measurements**, focusing in this example on mean magnitudes in the near-infrared K band. We restrict our attention to a specific table: `VVVX_VIRAC_V2_SOURCES`.

We begin by inspecting the column metadata of this table to locate photometry-related fields, specifically looking for UCDs such as `phot.mag;stat.mean;em.IR.K`, which indicates the mean K-band magnitude.

Using the identified metadata, we then construct a query to retrieve:
- the source identifier,
- astrometric positions (RA/Dec),
- astrometric uncertainties, and
- the K-band mean magnitude.

This approach ensures that we are using clearly described and standardized columns, as defined by the TAP service metadata, and allows for consistent extraction of scientifically meaningful data.

In [7]:
query = """
        SELECT column_name, datatype, unit, ucd, description
        FROM TAP_SCHEMA.columns
        WHERE table_name = 'VVVX_VIRAC_V2_SOURCES'
        """

result_table = tapcat.search(query).to_table()
result_table

column_name,datatype,unit,ucd,description
object,object,object,object,object
sourceid,BIGINT,,meta.id;meta.main,unique source identified
astfit_epochs,SMALLINT,,meta.number;time.epoch,number of epochs used for astrometric solution
astfit_params,SMALLINT,,meta.number,number of astrometric solution parameters
duplicate,SMALLINT,,meta.code,flag indicating a likely duplicate entry
ref_epoch,DOUBLE,yr,time.epoch,astrometric reference epoch
ra,DOUBLE,deg,pos.eq.ra;meta.main,right ascension
ra_error,DOUBLE,mas,stat.error;pos.eq.ra,uncertainty on right ascension
de,DOUBLE,deg,pos.eq.dec;meta.main,declination
de_error,DOUBLE,mas,stat.error;pos.eq.dec,uncertainty on declination


Filter for the UCD `phot.mag;stat.mean;em.IR.K` to find the mean K-band magnitude.

In [8]:
column_ks = result_table[result_table["ucd"] == "phot.mag;stat.mean;em.IR.K"]["column_name"][0]
column_ks

'phot_ks_mean_mag'

Now get the data for this table (including the source ID and astrometric positions).

In [9]:
query = f"""
        SELECT {column_id}, {column_ra}, {column_dec}, {column_ra_err}, {column_dec_err}, {column_ks}
        FROM VVVX_VIRAC_V2_SOURCES
        """

# result = tapcat.search(query, maxrec=100000).to_table() # to increase the number of rows, use maxrec=100000 or higher
result_table_data = tapcat.search(query).to_table()
result_table_data

sourceid,ra,de,ra_error,de_error,phot_ks_mean_mag
,deg,deg,mas,mas,mag
int64,float64,float64,float64,float64,float32
13550551016169,266.4084876728242,-29.149677083154415,8.046338335725856,7.383418271984562,16.778318
13550551006294,266.45124917744147,-29.165269592578042,9.570640695196355,8.100599445030499,15.640786
13550551008796,266.43861747586493,-29.143605072366665,5.838562117760776,4.45382184878046,16.272575
13550551000268,266.4103850308817,-29.14851364733599,6.5778333943788505,6.2211105022193625,16.572632
13550551013147,266.43465092359696,-29.171138624298848,9.933415976376736,9.44183225709199,16.937202
13550551005638,266.4653569344437,-29.14858784072673,7.502170093296878,6.82829031931015,16.820606
13550551005394,266.46160925899335,-29.142507185473548,3.743919010199219,4.780539055122571,16.242699
13550551005200,266.4720624656265,-29.147478268333966,13.58794958371984,12.583253966421948,17.016634


# **Query All Tables Containing K-band Photometric Information**

While the previous section focused on querying a specific table, here we broaden our scope to search **across all tables** published in the ESO TAP service for those that contain **K-band photometric data**.

We achieve this by querying the TAP metadata (`TAP_SCHEMA.columns` and `TAP_SCHEMA.tables`) for any column whose UCD includes the keyword `em.IR.K`, which corresponds to the near-infrared K band. This includes, for example, UCDs like `phot.mag;stat.mean;em.IR.K` (mean K-band magnitude) or other photometric descriptors associated with that wavelength.

The result is a list of all tables and columns across the archive that provide K-band photometry, allowing users to identify relevant datasets for broader studies in near-infrared astronomy.

This general approach is useful when:
- you're exploring multiple surveys,
- you want to build a cross-survey catalogue,
- or you're looking for all available sources with K-band measurements.

In [10]:
query = """
        SELECT collection, T.table_name, column_name, ucd
        FROM TAP_SCHEMA.columns as C, TAP_SCHEMA.tables as T
        WHERE T.table_name=C.table_name
        AND (ucd LIKE '%em.IR.K%')

        -- The ucd tells which columns are carrying the main coordinates and source identifier.
        """

result = tapcat.search(query).to_table()
result

collection,table_name,column_name,ucd
object,object,object,object
UltraVISTA,COSMOS2015_Laigle_v1_1b_latestV7_fits_V1,Ks_FLUX_APER2,phot.flux;em.IR.K
UltraVISTA,COSMOS2015_Laigle_v1_1b_latestV7_fits_V1,Ks_FLUXERR_APER2,stat.error;phot.flux;em.IR.K
UltraVISTA,COSMOS2015_Laigle_v1_1b_latestV7_fits_V1,Ks_FLUX_APER3,phot.flux;em.IR.K
UltraVISTA,COSMOS2015_Laigle_v1_1b_latestV7_fits_V1,Ks_FLUXERR_APER3,stat.error;phot.flux;em.IR.K
UltraVISTA,COSMOS2015_Laigle_v1_1b_latestV7_fits_V1,Ks_MAG_APER2,phot.mag;em.IR.K
UltraVISTA,COSMOS2015_Laigle_v1_1b_latestV7_fits_V1,Ks_MAGERR_APER2,stat.error;phot.mag;em.IR.K
UltraVISTA,COSMOS2015_Laigle_v1_1b_latestV7_fits_V1,Ks_MAG_APER3,phot.mag;em.IR.K
UltraVISTA,COSMOS2015_Laigle_v1_1b_latestV7_fits_V1,Ks_MAGERR_APER3,stat.error;phot.mag;em.IR.K
UltraVISTA,COSMOS2015_Laigle_v1_1b_latestV7_fits_V1,Ks_MAG_AUTO,phot.mag;em.IR.K
